<a target="_blank" href="https://colab.research.google.com/github/ZHAW-ZAV/TSO-FS25-students/blob/main/09_qpa/09_03_qpa3.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import sys
import os

IN_COLAB = "google.colab" in sys.modules

file_id_passenger_data = "1owvqkuVP_w918DGhgkL-DVqUWVf0iYrt"

if IN_COLAB:
    from google.colab import drive
    path_to_passenger_data = "/content/data/passenger_data.csv"
    os.makedirs(os.path.dirname(path_to_passenger_data), exist_ok=True)
    !gdown "https://drive.google.com/uc?id={file_id_passenger_data}" -O "{path_to_passenger_data}"

    # Download the TSO environment
    drive.mount("/content/drive")
    shared_file = "1fuyEnXBRBZJ1O2ijOLx4ms-L4si430RJ"
    !gdown "https://drive.google.com/uc?id={shared_file}" -O tso-env.zip

    !unzip tso-env.zip -d /content | tqdm > /dev/null

    !source /content/tso-env/bin/activate

    sys.path.append("/content/tso-env/lib/python3.11/site-packages/")
else:
    import gdown

    url = f"https://drive.google.com/uc?id={file_id_passenger_data}"
    path_to_passenger_data = "data/passenger_data.csv"
    os.makedirs(os.path.dirname(path_to_passenger_data), exist_ok=True)
    gdown.download(url, path_to_passenger_data, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1owvqkuVP_w918DGhgkL-DVqUWVf0iYrt
To: /Users/wate/Documents/GitHub/TSO-FS25-students/09_qpa/data/passenger_data.csv
100%|██████████| 359k/359k [00:00<00:00, 18.4MB/s]


In [2]:
import simpy

In [3]:
# Load passenger data from CSV
# df_passengers = pd.read_csv('data/passenger_data.csv')
df_passengers = pd.read_csv(path_to_passenger_data)

# Load passenger arrival times from df_passengers
df_passengers["timestamp"] = pd.to_datetime(df_passengers["timestamp"])  # Ensure datetime format

# Compute interarrival times in seconds
df_passengers["interarrival_time"] = df_passengers["timestamp"].diff().dt.total_seconds().fillna(0)

# Convert interarrival times to a list
interarrival_times = df_passengers["interarrival_time"].tolist()

The code above loads the data, do not modify.

To-DO:
* External File handling (arrival_times)

***

# TSO Semester Week 11: Quantitative Process Analysis

This exercise deals with the topic of quantitative process analysis using the example of a security check in a fictitious airport terminal. Use the DES simulation model of an M/M/c queueing system provided below for this exercise. In this exercise, you will apply the DES model to real-world input data describing the time of arrival of passengers at the security checkpoint (i.e. in contrast to the last two weeks, we no longer assume that the interarrival time can be described by a random process). The observed interarrival times of the passengers are stored in the list `interarrival_times`.

The simulation results of all `NUM_RUNS` simulation runs are stored in `df_queue_length`. The columns of this dataframe refer to the simulation run, the rows to the time in seconds after midnight.


## PART 1:
This exercise consists of only one PART, which has the following tasks:


### Tasks
1. Run the DES simulation model below.
2. Create a plot showing (i) the number of passengers entering the security checkpoint per 5 minute interval over the course of the day, and (ii) the simulated delay over the course of the day.
3. Determine the maximum simulated delay and determine the space required for the queue. Use the spatial target level of service `SP` (i.e., space per waiting passenger) from week 9.


In [ ]:
# PART 1, Task 1: DES Simulation Model

#  Simulation parameters
C = 4                               # Number of Servers
SERVICE_RATE = 1/5.8         # Mu: Average number of services per server per time unit
SIM_TIME = 17 * 3600                # Total simulation time [seconds]
NUM_RUNS = 100                      # Number of independent simulation runs
CUSTOMER_INTERVAL = 5               # Track every 5th customer
TIME_TRACK_INTERVAL = 60            # Track queue length every 60 seconds

# Function to generate service times
def generate_service_time(mean_service_rate):
    """Generate service time based on exponential distribution."""
    return np.random.exponential(1 / mean_service_rate)

# Simulation function
def simulate_one_run(interarrival_times):
    """Simulate a single M/M/c run and record queue length over time."""
    queue_length_over_time = {}

    def customer(env, name, server, service_rate):
        """A customer process arriving and waiting for service."""
        arrival_time = env.now

        # Log queue length at arrival
        queue_length_at_arrival = len(server.queue)

        # Record queue length at this time
        queue_length_over_time[env.now] = queue_length_at_arrival

        with server.request() as request:
            yield request  # Wait for the server
            service_time = generate_service_time(service_rate)
            yield env.timeout(service_time)  # Simulate service

    def arrival_process(env, server, service_rate, interarrival_times):
        """Process customers based on the interarrival times from df_passengers."""
        customer_id = 0
        for interarrival_time in interarrival_times:
            yield env.timeout(interarrival_time)
            customer_id += 1
            env.process(customer(env, customer_id, server, service_rate))

    def track_queue_length(env, server):
        """Track queue length at fixed time intervals."""
        while True:
            queue_length_over_time[env.now] = len(server.queue)
            yield env.timeout(TIME_TRACK_INTERVAL)

    # Create the simulation environment
    env = simpy.Environment()
    server = simpy.Resource(env, capacity=C)

    # Start processes
    env.process(arrival_process(env, server, SERVICE_RATE, interarrival_times))
    env.process(track_queue_length(env, server))

    env.run(until=SIM_TIME)

    return queue_length_over_time

# Run the simulation NUM_RUNS times and collect queue length data
queue_length_runs = []

for run in tqdm(range(NUM_RUNS), desc="Running Simulations", unit="run"):
    queue_length_over_time = simulate_one_run(interarrival_times)
    queue_length_runs.append(queue_length_over_time)

# Convert queue lengths to DataFrame
df_queue_length = pd.DataFrame(queue_length_runs).T

Running Simulations: 100%|██████████| 100/100 [00:06<00:00, 14.86run/s]


In [4]:
# PART 1, Task 1: Create df_inflow

# Create a copy to avoid modifying df_passengers
df_temp = df_passengers.copy()

# Set 'timestamp' as the index in the copied DataFrame
df_temp.set_index('timestamp', inplace=True)

# Resample into 5-minute intervals and count the passengers
df_inflow = df_temp.resample('5min').size().reset_index(name='inflow')

# Add 'seconds_from_midnight' to df_inflow
df_inflow['seconds_from_midnight'] = (
    df_inflow['timestamp'].dt.hour * 3600 +
    df_inflow['timestamp'].dt.minute * 60 +
    df_inflow['timestamp'].dt.second
)

df_inflow

,timestamp,inflow,seconds_from_midnight
0,2021-07-15 03:30:00,1,12600
1,2021-07-15 03:35:00,0,12900
2,2021-07-15 03:40:00,0,13200
3,2021-07-15 03:45:00,0,13500
4,2021-07-15 03:50:00,0,13800
...,...,...,...
226,2021-07-15 22:20:00,1,80400
227,2021-07-15 22:25:00,1,80700
228,2021-07-15 22:30:00,0,81000
229,2021-07-15 22:35:00,1,81300


In [4]:
# PART 1, Task 2: Plot Inflow and Queue Length over Time


In [5]:
# PART 1, Task 3: Determine Queue Size
